In [1]:
import pandas as pd
import numpy as np
import random
import string

# Set random seed for reproducibility
np.random.seed(42)

# Define the number of rows and columns
num_rows = 10_000_000  # 10 million rows
num_columns = 10       # 10 columns

# Helper function to generate random strings
def random_string(length=10):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(length))

# Create random data for each column
data = {
    "id": np.arange(1, num_rows + 1),                             # Unique ID
    "age": np.random.randint(18, 80, size=num_rows),              # Random age between 18 and 80
    "salary": np.random.uniform(30000, 120000, size=num_rows),     # Random float for salary
    "date_of_joining": pd.date_range('2000-01-01', periods=num_rows, freq='T'),  # Random dates
    "department": np.random.choice(['HR', 'Finance', 'IT', 'Sales'], size=num_rows),  # Random department
    "email": [random_string() + "@example.com" for _ in range(num_rows)],  # Random email strings
    "city": np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston'], size=num_rows),  # Cities
    "performance_score": np.random.randint(1, 10, size=num_rows),  # Performance score (1-10)
    "gender": np.random.choice(['Male', 'Female'], size=num_rows),  # Random gender
    "phone_number": ['+1-' + ''.join(np.random.choice(list('0123456789'), 10)) for _ in range(num_rows)],  # Random phone numbers
}

# Convert the dictionary into a pandas DataFrame
df = pd.DataFrame(data)

# Save to CSV (without index)
df.to_csv("large_employee_records.csv", index=False)

print("2GB dataset generated and saved as 'large_employee_records.csv'.")

2GB dataset generated and saved as 'large_employee_records.csv'.


In [3]:
import pandas as pd

# Read the CSV file in chunks to handle large size
chunk_size = 1_000_000
chunks = pd.read_csv("large_employee_records.csv", chunksize=chunk_size)

# Example: Process and combine chunks
df = pd.concat(chunks, ignore_index=True)
print(df.head())

   id  age         salary      date_of_joining department  \
0   1   56  105443.188596  2000-01-01 00:00:00         IT   
1   2   69   87745.794238  2000-01-01 00:01:00         HR   
2   3   46  108896.424998  2000-01-01 00:02:00         HR   
3   4   32  119826.816879  2000-01-01 00:03:00    Finance   
4   5   60   91009.982737  2000-01-01 00:04:00         HR   

                    email         city  performance_score  gender  \
0  ydzgrybrmk@example.com      Chicago                  6  Female   
1  qdmlctycoq@example.com      Chicago                  5  Female   
2  uukgzlztpf@example.com  Los Angeles                  4  Female   
3  fhefgoqeca@example.com     New York                  5    Male   
4  cpdpyarzks@example.com  Los Angeles                  7  Female   

    phone_number  
0  +1-7228155984  
1  +1-2029290494  
2  +1-9200789379  
3  +1-8350371798  
4  +1-6897024588  


In [4]:
import dask.dataframe as dd

# Read the CSV file with Dask
df_dask = dd.read_csv("large_employee_records.csv")

# Compute to load into memory (optional, based on your needs)
df = df_dask.compute()
print(df.head())

   id  age         salary      date_of_joining department  \
0   1   56  105443.188596  2000-01-01 00:00:00         IT   
1   2   69   87745.794238  2000-01-01 00:01:00         HR   
2   3   46  108896.424998  2000-01-01 00:02:00         HR   
3   4   32  119826.816879  2000-01-01 00:03:00    Finance   
4   5   60   91009.982737  2000-01-01 00:04:00         HR   

                    email         city  performance_score  gender  \
0  ydzgrybrmk@example.com      Chicago                  6  Female   
1  qdmlctycoq@example.com      Chicago                  5  Female   
2  uukgzlztpf@example.com  Los Angeles                  4  Female   
3  fhefgoqeca@example.com     New York                  5    Male   
4  cpdpyarzks@example.com  Los Angeles                  7  Female   

    phone_number  
0  +1-7228155984  
1  +1-2029290494  
2  +1-9200789379  
3  +1-8350371798  
4  +1-6897024588  


In [11]:
# Clean column names
df.columns = [col.strip().replace(' ', '_').lower() for col in df.columns]

# Optionally save the cleaned dataset
df.to_csv("large_employee_records_cleaned.csv", index=False)
print("Column names cleaned and dataset saved as 'large_employee_records_cleaned.csv'.")

Column names cleaned and dataset saved as 'large_employee_records_cleaned.csv'.


In [12]:
df.to_csv("large_employee_records.txt.gz", sep='|', compression='gzip', index=False)
print("File saved as 'large_employee_records.txt.gz' in pipe-separated format.")

File saved as 'large_employee_records.txt.gz' in pipe-separated format.


In [14]:
import os

# Read the pipe-separated text file
df_summary = pd.read_csv("large_employee_records.txt.gz", sep='|')

# Generate summary
total_rows = len(df_summary)
total_columns = len(df_summary.columns)
file_size = os.path.getsize("large_employee_records.txt.gz") / (1024 * 1024)  # Size in GB

print(f"Total number of rows: {total_rows}")
print(f"Total number of columns: {total_columns}")
print(f"File size: {file_size:.2f} GB")

Total number of rows: 10000000
Total number of columns: 10
File size: 345.98 GB


In [16]:
import yaml

# Define expected columns based on your dataset
expected_columns = df.columns.tolist()

# Create a schema dictionary
schema = {'columns': expected_columns}

# Write schema to a YAML file
with open('schema.yaml', 'w') as file:
    yaml.dump(schema, file, default_flow_style=False)

print("YAML schema created and saved as 'schema.yaml'.")

YAML schema created and saved as 'schema.yaml'.


In [17]:
import yaml

# Load the schema from the YAML file
with open('schema.yaml', 'r') as file:
    schema = yaml.safe_load(file)

expected_columns = schema['columns']

# Validate that the actual dataset columns match the expected schema
actual_columns = df.columns.tolist()

if set(expected_columns) == set(actual_columns):
    print("Column names match the schema.")
else:
    print("Column names do not match the schema.")
    print(f"Expected columns: {expected_columns}")
    print(f"Actual columns: {actual_columns}")

Column names match the schema.


In [18]:
import pandas as pd
import yaml

# Load your dataset
df = pd.read_csv("large_employee_records_cleaned.csv")

# Load the YAML schema file
with open('schema.yaml', 'r') as file:
    schema = yaml.safe_load(file)

# Extract the expected column names from the YAML schema
expected_columns = schema['columns']

# Extract the actual column names from the dataset
actual_columns = df.columns.tolist()

# Check if the actual columns match the expected columns
if set(expected_columns) == set(actual_columns):
    print("Column names in the dataset accurately match the YAML schema.")
else:
    print("Mismatch in columns!")
    print(f"Expected columns: {expected_columns}")
    print(f"Actual columns: {actual_columns}")
    
    # Find missing columns
    missing_in_data = set(expected_columns) - set(actual_columns)
    if missing_in_data:
        print(f"Columns in YAML but missing in dataset: {missing_in_data}")
    
    # Find extra columns
    extra_in_data = set(actual_columns) - set(expected_columns)
    if extra_in_data:
        print(f"Extra columns in dataset not in YAML: {extra_in_data}")

Column names in the dataset accurately match the YAML schema.


In [19]:
import pandas as pd
import yaml

# Load your dataset
df = pd.read_csv("large_employee_records_cleaned.csv")

# Load the YAML schema file
with open('schema.yaml', 'r') as file:
    schema = yaml.safe_load(file)

# Extract the expected column names from the YAML schema
expected_columns = schema['columns']

# Extract the actual column names from the dataset
actual_columns = df.columns.tolist()

# Validate column names
if expected_columns == actual_columns:
    print("Success: Column names in the dataset match the schema exactly.")
else:
    print("Error: Column names do not match the schema.")
    
    # Find missing or extra columns
    missing_columns = set(expected_columns) - set(actual_columns)
    extra_columns = set(actual_columns) - set(expected_columns)

    if missing_columns:
        print(f"Missing columns in dataset: {missing_columns}")
    
    if extra_columns:
        print(f"Extra columns in dataset: {extra_columns}")

Success: Column names in the dataset match the schema exactly.


In [31]:
print(df.head())

   id  age         salary      date_of_joining department  \
0   1   56  105443.188596  2000-01-01 00:00:00         IT   
1   2   69   87745.794238  2000-01-01 00:01:00         HR   
2   3   46  108896.424998  2000-01-01 00:02:00         HR   
3   4   32  119826.816879  2000-01-01 00:03:00    Finance   
4   5   60   91009.982737  2000-01-01 00:04:00         HR   

                    email         city  performance_score  gender  \
0  ydzgrybrmk@example.com      Chicago                  6  Female   
1  qdmlctycoq@example.com      Chicago                  5  Female   
2  uukgzlztpf@example.com  Los Angeles                  4  Female   
3  fhefgoqeca@example.com     New York                  5    Male   
4  cpdpyarzks@example.com  Los Angeles                  7  Female   

    phone_number  
0  +1-7228155984  
1  +1-2029290494  
2  +1-9200789379  
3  +1-8350371798  
4  +1-6897024588  
